# Machine Learning model for Credit Card Fraud Detection

### Importing the libraries

In [101]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

### Loading the dataset

In [102]:
data = pd.read_csv('creditcard.csv')

In [103]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [104]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [105]:
data['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [106]:
# separating data for analysis
legit = data[data.Class == 0]  #normal transaction
fraud = data[data.Class == 1]  #fraud transaction

In [107]:
print(legit.shape)
print(fraud.shape)

(284315, 31)
(492, 31)


In [108]:
legit.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [109]:
fraud.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [110]:
data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


Build a sample dataset with similar similar distribution of normal and fraud transaction

In [111]:
legit_sample = legit.sample(n=492)

In [112]:
# concatenating two dataframes
new = pd.concat([legit_sample, fraud], axis=0)
new.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
13637,24171.0,1.189648,0.142604,-0.047900,1.022025,0.703172,1.147797,-0.210647,0.185894,1.393414,...,-0.378896,-0.796104,-0.187757,-1.798686,0.612810,-0.418917,0.004890,-0.007002,35.80,0
123716,77034.0,-0.092220,0.395562,0.471270,-1.622755,-0.148865,-1.756109,0.739116,-0.376327,-1.430424,...,0.362319,1.083530,0.015539,0.747909,-0.660064,-0.445587,0.430581,0.283345,15.00,0
9294,13445.0,1.213253,0.088066,1.023482,0.227933,-0.775637,-0.664523,-0.443000,-0.097946,1.320945,...,-0.275215,-0.564594,0.175837,0.520234,-0.053007,0.718746,-0.088787,0.000870,2.49,0
231289,146690.0,-0.672693,1.495938,-0.765138,-1.563996,1.652712,-1.040669,2.122628,-0.889164,0.511979,...,0.020732,0.988039,-0.297722,0.804643,-0.195167,-0.035425,0.310536,-0.142742,1.16,0
280859,169794.0,-0.286986,1.422944,-0.846387,-0.780135,1.188448,-0.256113,0.682947,-0.841935,0.181215,...,0.362416,-1.168507,0.141286,-0.084239,-0.269353,0.164490,0.368495,0.147121,0.89,0


In [113]:
new['Class'].value_counts()

1    492
0    492
Name: Class, dtype: int64

In [114]:
new.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,95080.296748,0.014262,0.042817,-0.025309,-0.051061,0.053509,0.004026,0.077534,-0.050543,0.046540,...,-0.003302,0.007810,0.014017,-0.000003,-0.038264,0.000890,0.014611,-0.000229,0.004316,86.471687
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [115]:
x = new.drop(columns='Class', axis=1)
y = new['Class']
print(x)

            Time        V1        V2        V3        V4        V5        V6  \
13637    24171.0  1.189648  0.142604 -0.047900  1.022025  0.703172  1.147797   
123716   77034.0 -0.092220  0.395562  0.471270 -1.622755 -0.148865 -1.756109   
9294     13445.0  1.213253  0.088066  1.023482  0.227933 -0.775637 -0.664523   
231289  146690.0 -0.672693  1.495938 -0.765138 -1.563996  1.652712 -1.040669   
280859  169794.0 -0.286986  1.422944 -0.846387 -0.780135  1.188448 -0.256113   
...          ...       ...       ...       ...       ...       ...       ...   
279863  169142.0 -1.927883  1.125653 -4.518331  1.749293 -1.566487 -2.010494   
280143  169347.0  1.378559  1.289381 -5.004247  1.411850  0.442581 -1.326536   
280149  169351.0 -0.676143  1.126366 -2.213700  0.468308 -1.120541 -0.003346   
281144  169966.0 -3.113832  0.585864 -5.399730  1.817092 -0.840618 -2.943548   
281674  170348.0  1.991976  0.158476 -2.583441  0.408670  1.151147 -0.096695   

              V7        V8        V9  .

In [116]:
print(y)

13637     0
123716    0
9294      0
231289    0
280859    0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64


### Split the data

In [117]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=2)

In [118]:
print(x.shape, x_train.shape, x_test.shape)

(984, 30) (787, 30) (197, 30)


### Model Training

In [119]:
model = LogisticRegression()

In [120]:
model.fit(x_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [121]:
train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(train_prediction, y_train)

In [122]:
print('Accuracy on Training data: ', training_data_accuracy)

Accuracy on Training data:  0.9415501905972046


In [123]:
test_prediction = model.predict(x_test)
test_data_accuracy = accuracy_score(test_prediction, y_test)

In [124]:
print('Accuracy score on Test data: ', test_data_accuracy)

Accuracy score on Test data:  0.9238578680203046
